In [1]:
from transformers import AutoTokenizer
model_path = '/Users/zhangyf/llm/gpt2'
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [2]:
from datasets import load_dataset
dataset_path = './sst2'
dataset = load_dataset(dataset_path)
print(dataset)

ds_train, ds_val = dataset['train'], dataset['validation']
print(ds_train[4])

DatasetDict({
    train: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 872
    })
    test: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 1821
    })
})
{'idx': 4, 'sentence': 'on the worst revenge-of-the-nerds clichés the filmmakers could dredge up ', 'label': 0}


In [6]:
# 定义一个新的token，奖励token eos
REWARD_TOKEN_ID = tokenizer.eos_token_id
print(REWARD_TOKEN_ID)

def tokenize(batch):
    # 提取出文本内容
    outputs = tokenizer(batch['sentence'])
    # 每条数据一个评分，初始化为 0 。
    outputs['score'] = [0] * len(outputs['input_ids'])
    # 对每条数据的最后的reward token进行评分
    outputs['score_index'] = [0] * len(outputs['input_ids'])
    for i in range(len(outputs['input_ids'])):
        # 第 i 条数据的末尾添加一个 eos token，作为reward token
        outputs['input_ids'][i].append(REWARD_TOKEN_ID)
        # reward token的掩码设置为 1 。
        outputs['attention_mask'][i].append(1)
        # 正向情感的文本评分为 1 。负向情感的评分为 0 。
        outputs['score'][i] = float(batch['label'][i])
        # 对 reward token 进行评分，也就是评分的索引为 reward token 的索引。
        outputs['score_index'][i] = len(outputs['input_ids'][i]) - 1
    return outputs

map_kwargs = {
    "batched": True,
    "batch_size": 512,
    "remove_columns": ['idx', 'sentence', 'label']
}

tokenized_dataset_train = ds_train.map(tokenize, **map_kwargs)
tokenized_dataset_val = ds_val.map(tokenize, **map_kwargs)

print(tokenized_dataset_train[4])

50256
{'input_ids': [261, 262, 5290, 15827, 12, 1659, 12, 1169, 12, 1008, 9310, 35478, 20954, 262, 28303, 714, 47478, 469, 510, 220, 50256], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'score': 0.0, 'score_index': 20}


In [7]:
tokenized_dataset_train.set_format(type='torch')
tokenized_dataset_val.set_format(type='torch')

print(tokenized_dataset_train[4])

{'input_ids': tensor([  261,   262,  5290, 15827,    12,  1659,    12,  1169,    12,  1008,
         9310, 35478, 20954,   262, 28303,   714, 47478,   469,   510,   220,
        50256]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), 'score': tensor(0.), 'score_index': tensor(20)}


In [8]:
tokenized_dataset_train = tokenized_dataset_train.filter(lambda x: len(x['input_ids']) > 6)
tokenized_dataset_val = tokenized_dataset_val.filter(lambda x: len(x['input_ids']) > 6)

print(len(tokenized_dataset_train))

Filter:   0%|          | 0/67349 [00:00<?, ? examples/s]

Filter:   0%|          | 0/872 [00:00<?, ? examples/s]

49401


In [9]:
import torch
from torch import nn
import numpy as np
from transformers import AutoModelForCausalLM

class RewardHead(nn.Module):
    def __init__(self, config):
        super().__init__()
        # llm最后输出的隐藏层的维度
        self.hidden_size = config.hidden_size
        # 线性层用来对llm最后输出的隐藏层给奖励
        self.reward = nn.Linear(self.hidden_size, 1)
        self._post_init()

    def _post_init(self):
        # 使用正态分布初始化权重
        nn.init.normal_(
            self.reward.weight,
            std=(1.0 / np.sqrt(self.hidden_size + 1))
        )
        # 将偏置初始化为0
        nn.init.zeros_(self.reward.bias)

    def forward(self, hidden_states):
        # 给出奖励
        return self.reward(hidden_states)

class GPT2RewardHead(nn.Module):
    def __init__(self, model_name):
        super().__init__()
        self.llm = AutoModelForCausalLM.from_pretrained(model_name)
        self.reward_head = RewardHead(self.llm.config)

    def forward(self, input_ids, attention_mask):
        transformer_outputs = self.llm.forward(
            input_ids=input_ids,
            attention_mask=attention_mask,
            output_hidden_states=True
        )
        last_hidden_state = transformer_outputs.hidden_states[-1]
        # 给出奖励
        reward = self.reward_head(last_hidden_state).squeeze(-1)
        # sigmoid用来将奖励搞到(0,1)范围内
        return torch.sigmoid(reward)

In [10]:
model = GPT2RewardHead(model_path)

In [11]:
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding
# 还是将 eos token 作为 pad token
tokenizer.pad_token = tokenizer.eos_token

data_collator = DataCollatorWithPadding(tokenizer)
dataloader_params = {
    'batch_size': 32,
    'shuffle': True,
    'collate_fn': data_collator
}
train_dataloader = DataLoader(tokenized_dataset_train, **dataloader_params)
val_dataloader = DataLoader(tokenized_dataset_val, **dataloader_params)

batch = next(iter(train_dataloader))
print(batch.keys())

print(batch['input_ids'][1])
print(batch['attention_mask'][1])
print(batch['score'][1])
print(batch['score_index'][1])
print(tokenizer.decode(batch['input_ids'][1]))
print(batch['attention_mask'][1].nonzero()[-1])

KeysView({'input_ids': tensor([[  372, 15347,   837,  ..., 50256, 50256, 50256],
        [ 6667, 28079,   284,  ..., 50256, 50256, 50256],
        [   69,  3289,  1838,  ..., 50256, 50256, 50256],
        ...,
        [  505,   286,   262,  ..., 50256, 50256, 50256],
        [  283,   380,  1158,  ..., 50256, 50256, 50256],
        [ 2339,   262,   670,  ...,  1296,   220, 50256]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1]]), 'score': tensor([1., 1., 0., 1., 0., 0., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1., 0., 1.,
        0., 1., 0., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 0.]), 'score_index': tensor([ 6, 28, 23,  6, 16, 18, 11,  8,  8, 12, 10, 26, 18, 11, 17, 11, 20,  9,
         7, 26, 11, 17,  8, 12,  9, 11, 11, 27, 13,  9, 21, 34])})
tensor([ 6667, 28079,   284,   262, 43937,  3326,   646,   8

/opt/anaconda3/envs/rlhf/lib/python3.10/site-packages/transformers/utils/generic.py:278: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/opt/anaconda3/envs/rlhf/lib/python3.10/site-packages/transformers/utils/generic.py:278: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.array(obj)


In [16]:
outputs = model(batch['input_ids'], batch['attention_mask'])
print(outputs.shape)

torch.Size([32, 35])


In [17]:
device = torch.device('mps') if torch.mps.is_available() else torch.device('cpu')
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
# 二分类交叉熵损失
criterion = nn.BCELoss()
num_epochs = 1 # N+ Implementation Detail paper

In [18]:
def validate():
    model.eval()
    total_loss = 0
    for i, batch in enumerate(val_dataloader):
        inputs = batch.to(device)
        model_inputs = {
            'input_ids': inputs['input_ids'],
            'attention_mask': inputs['attention_mask']
        }
        with torch.no_grad():
            # 对输出进行评分
            scores = model(**model_inputs)
            # 批次中每条数据的索引
            batch_indices = torch.arange(scores.shape[0])
            # 根据索引拿出评分，也就是reward token的评分
            score = scores[batch_indices, inputs['score_index']]
            # 目标评分，0 或者 1 。
            target = inputs['score']
            # 计算误差
            loss = criterion(score, target)
        total_loss += loss.item()
    print('validation loss:', total_loss / len(val_dataloader))

In [19]:
model.to(device)

validate()
for epoch in range(num_epochs):
    model.train()
    for i, batch in enumerate(train_dataloader):
        inputs = batch.to(device)
        model_inputs = {
            'input_ids': inputs['input_ids'],
            'attention_mask': inputs['attention_mask']
        }
        scores = model(**model_inputs)
        batch_indices = torch.arange(scores.shape[0])
        score = scores[batch_indices, inputs['score_index']]
        target = inputs['score']
        loss = criterion(score, target)
        # 三部曲：清空梯度 ⟶ 反向传播计算梯度 ⟶ 更新参数
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(loss.item())
    validate()

/opt/anaconda3/envs/rlhf/lib/python3.10/site-packages/transformers/utils/generic.py:278: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/opt/anaconda3/envs/rlhf/lib/python3.10/site-packages/transformers/utils/generic.py:278: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.array(obj)


validation loss: 4.79311728477478
4.4001851081848145
2.327467918395996
1.051131010055542
0.7866150140762329
0.7303826212882996
0.7172989845275879
0.6597344875335693
0.6107072234153748
0.7599384784698486
0.7308290004730225
0.7243120670318604
0.6095142364501953
0.7216007709503174
0.7142904996871948
0.7092389464378357
0.6783279180526733
0.6698957681655884
0.6810418367385864
0.77931809425354
0.6285977363586426
0.6507300138473511
0.633979320526123
0.6578609943389893
0.6933450698852539
0.617280125617981
0.6383077502250671
0.742990255355835
0.6310871839523315
0.6538606882095337
0.5633728504180908
0.6112344264984131
0.5538593530654907
0.7305097579956055
0.6105281114578247
0.5183444023132324
0.46167466044425964
0.710925817489624
0.4684641659259796
0.42701542377471924
0.5129765272140503
0.513247013092041
0.3424539268016815
0.628148078918457
0.7072775363922119
0.38046884536743164
0.4732533395290375
0.4835391044616699
0.30638059973716736
0.4838194251060486
0.3494548201560974
0.31996995210647583
0.

In [20]:
torch.save(model.state_dict(), 'reward_model.pt')

In [21]:
validate()

validation loss: 0.3069670636136185


In [23]:
from sklearn.metrics import confusion_matrix
model.eval()

all_predictions = []
all_labels = []

for i, batch in enumerate(val_dataloader):
    inputs = batch.to(device)
    model_inputs = {
        'input_ids': inputs['input_ids'],
        'attention_mask': inputs['attention_mask']
    }
    with torch.no_grad():
        scores = model(**model_inputs)
        batch_indices = torch.arange(scores.shape[0])
        score = scores[batch_indices, inputs['score_index']]
        target = inputs['score']
    predictions = (score > 0.5).int()

    all_predictions.extend(predictions.cpu().numpy())
    all_labels.extend(target.cpu().numpy())

confusion_matrix(all_labels, all_predictions)

/opt/anaconda3/envs/rlhf/lib/python3.10/site-packages/transformers/utils/generic.py:278: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/opt/anaconda3/envs/rlhf/lib/python3.10/site-packages/transformers/utils/generic.py:278: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.array(obj)


array([[375,  49],
       [ 40, 403]])